# Limpieza de Datos con Pandas

Este notebook demuestra técnicas comunes de limpieza de datos usando Pandas.

**Referencia:** [Limpieza de Datos](../pandas/python-para-datos/03-limpieza-datos.md)

**Objetivos:**
- Detectar problemas (nulos, duplicados, outliers)
- Manejar valores nulos
- Eliminar duplicados
- Corregir tipos de datos
- Normalizar y estandarizar datos
- Usar datos reales del CSV de ejemplo

## 1. Importar librerías

In [ ]:
import pandas as pd
import numpy as np

print("✅ Librerías importadas")

## 2. Cargar datos del CSV de ejemplo

In [ ]:
# Cargar datos desde el CSV de ejemplo
df = pd.read_csv('../data/ventas.csv')

print(f"✅ Datos cargados: {len(df)} registros")
print(f"\nShape: {df.shape}")
print(f"Columnas: {df.columns.tolist()}")
print("\nPrimeras filas:")
df.head()

## 3. Detectar problemas

In [ ]:
print("=== DETECCIÓN DE PROBLEMAS ===")

# 1. Valores nulos
print("\n1. Valores nulos por columna:")
nulos = df.isnull().sum()
print(nulos)
print(f"\nTotal de nulos: {nulos.sum()}")
if nulos.sum() > 0:
    print("Porcentaje de nulos:")
    print((nulos / len(df) * 100).round(2))
else:
    print("✅ No hay valores nulos")

# 2. Duplicados
print("\n2. Duplicados:")
duplicados_completos = df.duplicated().sum()
print(f"Duplicados completos: {duplicados_completos}")
duplicados_producto_fecha = df.duplicated(subset=['producto', 'fecha']).sum()
print(f"Duplicados en producto+fecha: {duplicados_producto_fecha}")
if duplicados_completos == 0 and duplicados_producto_fecha == 0:
    print("✅ No hay duplicados")

# 3. Tipos de datos
print("\n3. Tipos de datos:")
print(df.dtypes)
print("\nVerificando si 'fecha' es datetime:")
print(f"Tipo de 'fecha': {df['fecha'].dtype}")
if df['fecha'].dtype == 'object':
    print("⚠️ 'fecha' debería ser datetime, no texto")

# 4. Outliers en precio
print("\n4. Detección de outliers en 'precio':")
Q1 = df['precio'].quantile(0.25)
Q3 = df['precio'].quantile(0.75)
IQR = Q3 - Q1
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR
outliers = df[(df['precio'] < limite_inferior) | (df['precio'] > limite_superior)]
print(f"Outliers detectados: {len(outliers)}")
if len(outliers) > 0:
    print(outliers[['producto', 'precio', 'categoria']])

## 4. Limpiar datos

Aunque el CSV de ejemplo está relativamente limpio, aplicaremos técnicas de limpieza para demostrar el proceso.

In [ ]:
# Copiar para no modificar original
df_clean = df.copy()

print("=== PROCESO DE LIMPIEZA ===")
print(f"Registros iniciales: {len(df_clean)}")
print(f"Columnas: {df_clean.columns.tolist()}")

### 4.1 Convertir tipos de datos

In [ ]:
# Convertir fecha a datetime
df_clean['fecha'] = pd.to_datetime(df_clean['fecha'], format='%Y-%m-%d', errors='coerce')
print(f"✅ Fecha convertida a datetime: {df_clean['fecha'].dtype}")

# Convertir a categoría (ahorra memoria)
df_clean['categoria'] = df_clean['categoria'].astype('category')
df_clean['ciudad'] = df_clean['ciudad'].astype('category')
print(f"✅ Categorías convertidas a tipo 'category'")

print("\nTipos después de conversión:")
print(df_clean.dtypes)

### 4.2 Normalizar texto

In [ ]:
# Eliminar espacios al inicio y final
df_clean['producto'] = df_clean['producto'].str.strip()
df_clean['categoria'] = df_clean['categoria'].str.strip()

print("✅ Texto normalizado (espacios eliminados)")
print("\nEjemplo de productos normalizados:")
print(df_clean[['producto', 'categoria']].head())

### 4.3 Eliminar duplicados

In [ ]:
# Eliminar duplicados completos (si los hay)
registros_antes = len(df_clean)
df_clean = df_clean.drop_duplicates()
registros_despues = len(df_clean)
print(f"Registros antes: {registros_antes}")
print(f"Registros después: {registros_despues}")
print(f"Duplicados eliminados: {registros_antes - registros_despues}")

# Eliminar duplicados en producto+fecha (mantener el primero)
df_clean = df_clean.drop_duplicates(subset=['producto', 'fecha'], keep='first')
print(f"Registros después de eliminar duplicados en producto+fecha: {len(df_clean)}")

### 4.4 Manejar valores nulos (si los hubiera)

In [ ]:
# Verificar nulos
nulos = df_clean.isnull().sum()
print("Valores nulos después de limpieza:")
print(nulos)

if nulos.sum() > 0:
    print("\nOpciones para manejar nulos:")
    print("1. Eliminar filas con nulos: df.dropna()")
    print("2. Rellenar numéricos con promedio: df['columna'].fillna(df['columna'].mean())")
    print("3. Rellenar categóricos con valor fijo: df['columna'].fillna('Desconocido')")
    print("4. Forward fill: df.fillna(method='ffill')")
else:
    print("✅ No hay valores nulos que manejar")

### 4.5 Manejar outliers (opcional)

Si detectamos outliers, podemos eliminarlos o caparlos.

In [ ]:
# Detectar outliers en precio
Q1 = df_clean['precio'].quantile(0.25)
Q3 = df_clean['precio'].quantile(0.75)
IQR = Q3 - Q1
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

outliers = df_clean[(df_clean['precio'] < limite_inferior) | (df_clean['precio'] > limite_superior)]
print(f"Outliers detectados en precio: {len(outliers)}")

if len(outliers) > 0:
    print("\nOpciones:")
    print("1. Eliminar outliers: df = df[(df['precio'] >= limite_inferior) & (df['precio'] <= limite_superior)]")
    print("2. Capar outliers: df['precio'] = df['precio'].clip(lower=limite_inferior, upper=limite_superior)")
    print("\nOutliers encontrados:")
    print(outliers[['producto', 'precio', 'categoria']])
else:
    print("✅ No hay outliers significativos")

## 5. Verificar limpieza

In [ ]:
print("=== VERIFICACIÓN FINAL ===")
print(f"\nRegistros originales: {len(df)}")
print(f"Registros después de limpieza: {len(df_clean)}")
print(f"Registros eliminados: {len(df) - len(df_clean)}")

print("\nValores nulos restantes:")
nulos_finales = df_clean.isnull().sum()
print(nulos_finales)
if nulos_finales.sum() == 0:
    print("✅ No hay valores nulos")

print("\nDuplicados restantes:")
duplicados_finales = df_clean.duplicated().sum()
print(f"Duplicados completos: {duplicados_finales}")
if duplicados_finales == 0:
    print("✅ No hay duplicados")

print("\nTipos de datos:")
print(df_clean.dtypes)

print("\nDatos limpios (primeras filas):")
df_clean.head()

## 6. Resumen de limpieza

Este ejemplo muestra el proceso completo de limpieza de datos.

In [ ]:
print("=" * 60)
print("RESUMEN DE LIMPIEZA")
print("=" * 60)
print(f"\n✅ Tipos de datos corregidos:")
print(f"   - Fecha convertida a datetime")
print(f"   - Categoría y ciudad convertidas a 'category'")
print(f"\n✅ Texto normalizado:")
print(f"   - Espacios eliminados en producto y categoria")
print(f"\n✅ Duplicados verificados:")
print(f"   - Duplicados completos: {df.duplicated().sum()}")
print(f"   - Duplicados en producto+fecha: {df.duplicated(subset=['producto', 'fecha']).sum()}")
print(f"\n✅ Valores nulos verificados:")
print(f"   - Total de nulos: {df.isnull().sum().sum()}")
print(f"\n✅ Outliers detectados:")
Q1 = df['precio'].quantile(0.25)
Q3 = df['precio'].quantile(0.75)
IQR = Q3 - Q1
outliers = df[(df['precio'] < Q1 - 1.5*IQR) | (df['precio'] > Q3 + 1.5*IQR)]
print(f"   - Outliers en precio: {len(outliers)}")
print(f"\n✅ Registros finales: {len(df_clean)}")
print("=" * 60)